Import packages

In [2]:
import mne
import pandas as pd
from tqdm import tqdm
import warnings
import os
import re
warnings.filterwarnings("ignore", category=DeprecationWarning)


Extract baseline data

In [13]:

def extract_baseline_data(path_2_csv_files: str, path_2_save_baseline_file: str, labelsequence: int =1, bad_files: list =[]):
    """ 
    Extract baseline data from raw EEG file (*.csv) that was obtained from hyperscanning2-redesign experiment \n

    Arguments :
        - path_2_csv_files (str) : path to raw EEG file \n
        - path_2_save_baseline_file (str) : path to save extracted baseline file for each condition \n
        - labelsequence (int) : order of pre-defined label sequence, 1 (averted) is default \n
        - bad_files (list) (optional) : file name of raw EEG file, e.g., EEG-S8.csv, that wants to be skipped to process

    Return :
        Extracted *.fif (MNE-Python) file for each condition of hand (finger pointing and tracking). 
        There are 6 files in total for each participant.
    """

    list_file_names = []
    full_path_2_each_file = []
    # bad_files=["EEG-S8.csv"] # Hard coded for now because bad EEG file is inside a folder of EEG. Remove this later

    for (root, dirs, file) in os.walk(path_2_csv_files):
        for f in file:

            if ".csv" in f:

                if (f in bad_files):
                    
                    # Skip the bad file to be processed
                    print(f"Skipped bad file : {f}")
                    continue

                else:
                    # Populate all file names only
                    list_file_names.append(f)
                    list_file_names.sort()

                    # Populate all full paths of each filename
                    full_path_2_each_file.append(os.path.join(root, f))
                    full_path_2_each_file.sort()

            

        # Iterate all file names

        for i in tqdm(range(len(full_path_2_each_file)), desc="In progress"):

            try:
                labelsequence = int(labelsequence)

            except IOError as err_filename:
                print("The format of file name is not correct or file doesn't exist \nThe format must be 'EEG-Sx.csv' , x=subject number ")
                raise
            except ValueError as err_integer:
                print("The labelsequence input must be integer : ", err_integer)
                raise

            else:
                if  labelsequence < 1 or labelsequence > 12:
                    print("The value for labelsequence parameter is out of range. It must be be between 1 and 12")
                    raise IndexError
                else:

                    # Load the data
                    fileName = full_path_2_each_file[i]
                    print("Processing file : " + list_file_names[i])

                    # Read each file by using pandas
                    df = pd.read_csv(fileName, delimiter=',')
                    # %% Define columns for raw csv file
                    df.columns = ['Index', 'FP1', 'FP2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1',
                                    'O2', 'X1', 'X2', 'X3', 'X4', 'X5',
                                    'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'Marker']
                    
                    # Converting all markers (pandas data frame to list)
                    markers = df['Marker'].tolist()
                    
                    # %%  Find all experimental markers and print them out.
                    indicesOfMarkers = []  # Empty list to contain indices of markers
                    for i, c in enumerate(markers):
                        if "9999999" in str(c) : 
                            indicesOfMarkers.append(i) 
                    try:
                        number_markers = len(indicesOfMarkers)
                        if number_markers != 48:   # check if the number of markers = 48
                            raise ValueError("The {} file has incorrect number of markers : {} ! It MUST be 48".format(fileName,number_markers))
                    except ValueError as err_unmatch_markers:
                        print(err_unmatch_markers)
                        raise


                    # Create a list of labels for baseline data. We used only averted eye condition in UNITY.
                    # It actually does not matter for different eye condition because participant only sees a white screen during the baseline condition)

                    # Order = 1 (Averted) Odd subject no. For example, 1, 3, 5, etc.
                    oddOrder1 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                                    "averted_post_right_point", "averted_post_left_point"]

                    # Order = 1 (Averted) Even subject no. For example, 2, 4, 6, etc.
                    evenOrder1 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                                    "averted_post_left_point", "averted_post_right_point"]


                    # Put all labels into a list for baseline data
                    listOfOrders = []
                    listOfOrders.append(oddOrder1)
                    listOfOrders.append(evenOrder1)

                    # Number that is used to take the label (oddOrder1 atau evenOrder1)
                    i_label_taker = 0

                    if i % 2 == 0:

                        # Even number
                        i_label_taker = 0

                    else:

                        # Odd number
                        i_label_taker = 1
                   
                    chosenOrder = listOfOrders[i_label_taker]


                    # Get the first 12 markers' indices and extract the data
                    indicesofBaselineMarkers = indicesOfMarkers[:13]

                    # Get the 1st and 12th index and chunk dataframe based on those indices, and convert it into numpy array
                    # For some data, it can be 13 markers after being extracted because when we combined the data the markers of beginning are right after the closing marker


                    # %% Chunk the data based on opening and closing markers and get only the 16 channels data (columns)
                    chunkedData = []
                    for i in range(0, 12, 2):

                        chunkedData.append(df.iloc[indicesofBaselineMarkers[i] : indicesofBaselineMarkers[i+1], 1:17].to_numpy() * 1e-6)

                    # Load each baseline file into MNE Python (averted eye condition only for baseline)

                    # Create 16 channels montage 10-20 international standard
                    montage = mne.channels.make_standard_montage('standard_1020')

                    # Pick only 16 channels that are used in Cyton+Daisy OpenBCI
                    # Create info
                    ch_names = ['FP1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1', 'O2']
                    ch_types = ['eeg'] * 16
                    info = mne.create_info(
                        ch_names=ch_names,
                        sfreq=125,
                        ch_types=ch_types)
                    info.set_montage('standard_1020', match_case=False)
                
                    # Match pattern EEG-Sx (x = any number)
                    regex = r"\D{3}-S\d+"

                    # Create filename that will be used for each condition. There are 6 conditions. See oddOrder1 or evenOrder1
                    extracted_file_name_4_baseline = []
                    for i in chosenOrder:
                        extracted_file_name = re.search(regex,fileName)
                        extracted_file_name_4_baseline.append(fileName[extracted_file_name.start() : extracted_file_name.end()] + "-" + i + "_raw.fif")


                    # Save into *.fif files
                    for i, val in tqdm(enumerate(chunkedData), desc = "Saving process..."):
                        # Load data into MNE-Python
                        baseline_data_needs_label = mne.io.RawArray(val.transpose(), info, verbose=False)
                        # Define a folder where we want to save the baseline data
                        os.chdir(path_2_save_baseline_file) 
                        # Save the data in MNE format
                        baseline_data_needs_label.save(extracted_file_name_4_baseline[i], overwrite=True)


    print(f"All baseline files have been saved in fif format in this path {path_2_save_baseline_file}")


Running the function of extract_baseline_data

In [14]:
# TODO Uncomment this when checked
labelsequence = 1

# TODO Uncomment this when checked
path_2_csv_files = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG"

# TODO Uncomment this when checked
path_2_save_baseline_file = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data"

extract_baseline_data(path_2_csv_files, path_2_save_baseline_file, labelsequence)

In progress:   0%|          | 0/12 [00:00<?, ?it/s]

Processing file : EEG-S1.csv


Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_pre_left_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_pre_right_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_right_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S1-averted_left_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-re

Saving process...: 6it [00:00, 54.52it/s]
In progress:   8%|▊         | 1/12 [00:01<00:18,  1.66s/it]

Processing file : EEG-S10.csv


...

Saving process...: 6it [00:00, 54.40it/s]
In progress:  92%|█████████▏| 11/12 [00:14<00:01,  1.33s/it]

Processing file : EEG-S9.csv


Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S9-averted_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S9-averted_pre_left_point_raw.fif
[done]
Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S9-averted_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S9-averted_pre_right_point_raw.fif
[done]
Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S9-averted_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S9-averted_right_tracking_raw.fif
[done]
Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S9-averted_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EE

Saving process...: 6it [00:00, 55.25it/s]
In progress: 100%|██████████| 12/12 [00:15<00:00,  1.33s/it]

All baseline files have been saved in fif format in this path /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data


Extract experimental data

In [15]:
def extract_experimental_data(path_2_csv_files: str, path_2_save_experimental_file: str, labelsequence_experiment: list, bad_files: list =[]):
    """ 
    Extract experimental data from raw EEG file (*.csv) that was obtained from hyperscanning2-redesign experiment \n

    Arguments :
        - path_2_csv_files (str) : path to raw EEG file \n
        - path_2_save_experimental_file (str) : path to save extracted baseline file for each condition \n
        - labelsequence (int) : order of pre-defined label sequence \n
        - bad_files (list) (optional) : file name(s) of raw EEG data, e.g., EEG-S8.csv, that wants to be skipped to process

    Return :
        Extracted *.fif (MNE-Python) file for each condition of hand (finger pointing and tracking). 
        There are 18 files in total for each participant.
    """

    list_file_names = []
    full_path_2_each_file = []
    # bad_files=["EEG-S8.csv"] # Hard coded for now because bad EEG file is inside a folder of EEG. Remove this later
    
    for (root, dirs, file) in os.walk(path_2_csv_files):
        for f in file:

            if ".csv" in f:

                if (f in bad_files):
                    
                    # Skip the bad file to be processed
                    print(f"Skipped bad file : {f}")
                    continue

                else:
                    # Populate all file names only
                    list_file_names.append(f)
                    list_file_names.sort()

                    # Populate all full paths of each filename
                    full_path_2_each_file.append(os.path.join(root, f))
                    full_path_2_each_file.sort()

            

        # Iterate all file names

        for i in tqdm(range(len(full_path_2_each_file)), desc="In progress"):

            try:
                labelsequence = int(labelsequence_experiment[i])

            except IOError as err_filename:
                print("The format of file name is not correct or file doesn't exist \nThe format must be 'EEG-Sx.csv' , x=subject number ")
                raise
            except ValueError as err_integer:
                print("The labelsequence input must be integer : ", err_integer)
                raise

            else:
                if  labelsequence < 1 or labelsequence > 12:
                    print("The value for labelsequence parameter is out of range. It must be be between 1 and 12")
                    raise IndexError
                else:

                    # Load the data
                    fileName = full_path_2_each_file[i]
                    print("Processing file : " + list_file_names[i])

                    # Read each file by using pandas
                    df = pd.read_csv(fileName, delimiter=',')
                    # Define columns for raw csv file
                    df.columns = ['Index', 'FP1', 'FP2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1',
                                    'O2', 'X1', 'X2', 'X3', 'X4', 'X5',
                                    'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'Marker']
                    
                    # Converting all markers (pandas data frame to list)
                    markers = df['Marker'].tolist()
                    
                    # Find all experimental markers and print them out.
                    indicesOfMarkers = []  # Empty list to contain indices of markers
                    for i, c in enumerate(markers):
                        if "9999999" in str(c) : 
                            indicesOfMarkers.append(i) 
                    try:
                        number_markers = len(indicesOfMarkers)
                        if number_markers != 48:   # check if the number of markers = 48
                            raise ValueError("The {} file has incorrect number of markers : {} ! It MUST be 48".format(fileName,number_markers))
                    except ValueError as err_unmatch_markers:
                        print(err_unmatch_markers)
                        raise


                    # Order = 1 (Averted/Direct/Natural)
                    oddOrder1 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                                "averted_post_right_point", "averted_post_left_point", "direct_pre_right_point", "direct_pre_left_point",
                                "direct_left_tracking", "direct_right_tracking", "direct_post_right_point", "direct_post_left_point",
                                "natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                                "natural_post_right_point", "natural_post_left_point"]

                    evenOrder1 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                                "averted_post_left_point", "averted_post_right_point", "direct_pre_left_point", "direct_pre_right_point",
                                "direct_right_tracking", "direct_left_tracking", "direct_post_left_point", "direct_post_right_point",
                                "natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                                "natural_post_left_point", "natural_post_right_point"]

                    # Order = 2 (Averted/Natural/Direct)
                    oddOrder2 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                                "averted_post_right_point", "averted_post_left_point", "natural_pre_right_point", "natural_pre_left_point",
                                "natural_left_tracking", "natural_right_tracking", "natural_post_right_point", "natural_post_left_point",
                                "direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                                "direct_post_right_point", "direct_post_left_point"]

                    evenOrder2 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                                "averted_post_left_point", "averted_post_right_point", "natural_pre_left_point", "natural_pre_right_point",
                                "natural_right_tracking", "natural_left_tracking", "natural_post_left_point", "natural_post_right_point",
                                "direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                                "direct_post_left_point", "direct_post_right_point"]

                    # Order = 3 (Direct / Natural / Averted)
                    oddOrder3 = ["direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                                "direct_post_right_point", "direct_post_left_point", "natural_pre_right_point", "natural_pre_left_point",
                                "natural_left_tracking", "natural_right_tracking", "natural_post_right_point", "natural_post_left_point",
                                "averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                                "averted_post_right_point", "averted_post_left_point"]

                    evenOrder3 = ["direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                                "direct_post_left_point", "direct_post_right_point", "natural_pre_left_point", "natural_pre_right_point",
                                "natural_right_tracking", "natural_left_tracking", "natural_post_left_point", "natural_post_right_point",
                                "averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                                "averted_post_left_point", "averted_post_right_point"]

                    # Order = 4 (Direct/Averted/Natural)
                    oddOrder4 = ["direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                                "direct_post_right_point", "direct_post_left_point", "averted_pre_right_point", "averted_pre_left_point",
                                "averted_left_tracking", "averted_right_tracking", "averted_post_right_point", "averted_post_left_point",
                                "natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                                "natural_post_right_point", "natural_post_left_point"]

                    evenOrder4 = ["direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                                "direct_post_left_point", "direct_post_right_point", "averted_pre_left_point", "averted_pre_right_point",
                                "averted_right_tracking", "averted_left_tracking", "averted_post_left_point", "averted_post_right_point",
                                "natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                                "natural_post_left_point", "natural_post_right_point"]

                    # Order = 5 (Natural/Direct/Averted)
                    oddOrder5 = ["natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                                "natural_post_right_point", "natural_post_left_point", "direct_pre_right_point", "direct_pre_left_point",
                                "direct_left_tracking", "direct_right_tracking", "direct_post_right_point", "direct_post_left_point",
                                "averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                                "averted_post_right_point", "averted_post_left_point"]

                    evenOrder5 = ["natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                                "natural_post_left_point", "natural_post_right_point", "direct_pre_left_point", "direct_pre_right_point",
                                "direct_right_tracking", "direct_left_tracking", "direct_post_left_point", "direct_post_right_point",
                                "averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                                "averted_post_left_point", "averted_post_right_point"]

                    # Order = 6 (Natural/Averted/Direct)
                    oddOrder6 = ["natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                                "natural_post_right_point", "natural_post_left_point", "averted_pre_right_point", "averted_pre_left_point",
                                "averted_left_tracking", "averted_right_tracking", "averted_post_right_point", "averted_post_left_point",
                                "direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                                "direct_post_right_point", "direct_post_left_point"]

                    evenOrder6 = ["natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                                "natural_post_left_point", "natural_post_right_point", "averted_pre_left_point", "averted_pre_right_point",
                                "averted_right_tracking", "averted_left_tracking", "averted_post_left_point", "averted_post_right_point",
                                "direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                                "direct_post_left_point", "direct_post_right_point"]


                    # Populate all orders
                    listOfOrders = []
                    for i in tqdm(range(6)):
                        listOfOrders.append(eval('oddOrder' + str(i+1)))
                        listOfOrders.append(eval('evenOrder' + str(i+1)))

                    
                    chosenOrder = listOfOrders[labelsequence-1]

                    # Chunk the data based on opening and closing markers and get only the 16 channels data (columns)
                    chunkedData = []
                    for i in range(0, 36, 2):

                        # Change into numpy and convert it from uV (microvolts) / nV to V (volts)
                        chunkedData.append(df.iloc[indicesOfMarkers[i]: indicesOfMarkers[i + 1] + 1, 1:17].to_numpy() * 1e-6)


                    # Get 12 markers' indices and extract experimental data
                    indicesofBaselineMarkers = indicesOfMarkers[13:]

                    # Create 16 channels montage 10-20 international standard
                    montage = mne.channels.make_standard_montage('standard_1020')

                    # Pick only 16 channels that are used in Cyton+Daisy OpenBCI
                    # Create info
                    ch_names = ['FP1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1', 'O2']
                    ch_types = ['eeg'] * 16
                    info = mne.create_info(
                        ch_names=ch_names,
                        sfreq=125,
                        ch_types=ch_types)
                    info.set_montage('standard_1020', match_case=False)
                
                    # Match pattern EEG-Sx (x = any number)
                    regex = r"\D{3}-S\d+"

                    # Create filename that will be used for each condition. There are 18 conditions for each participant.
                    extracted_file_name_4_baseline = []
                    for i in chosenOrder:
                        extracted_file_name = re.search(regex,fileName)
                        extracted_file_name_4_baseline.append(fileName[extracted_file_name.start() : extracted_file_name.end()] + "-" + i + "_raw.fif")


                    # Save into *.fif files
                    for i, val in tqdm(enumerate(chunkedData), desc = "Saving process..."):
                        # Load data into MNE-Python
                        baseline_data_needs_label = mne.io.RawArray(val.transpose(), info, verbose=False)
                        # Define a folder where we want to save the baseline data
                        os.chdir(path_2_save_experimental_file) 
                        # Save the data in MNE format
                        baseline_data_needs_label.save(extracted_file_name_4_baseline[i], overwrite=True)


    print(f"All experimental files have been saved in fif format in this path {path_2_save_experimental_file}")

Running the function of extract_experimental_data

In [16]:

# TODO Uncomment this when checked
labelsequence_hyper2_redesign = [1,2,1,2,1,2,3,4,3,4] 

# TODO Uncomment this when checked
path_2_csv_files = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG"

# TODO Uncomment this when checked
path_2_save_experimental_file = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data"


extract_experimental_data(path_2_csv_files, path_2_save_experimental_file, labelsequence_hyper2_redesign)

In progress:   0%|          | 0/12 [00:00<?, ?it/s]

Processing file : EEG-S1.csv


100%|██████████| 6/6 [00:00<00:00, 34904.06it/s]


Data have been extracted from : /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S1.csv

labelsequence is 1

Choseorder is ['averted_pre_right_point', 'averted_pre_left_point', 'averted_left_tracking', 'averted_right_tracking', 'averted_post_right_point', 'averted_post_left_point', 'direct_pre_right_point', 'direct_pre_left_point', 'direct_left_tracking', 'direct_right_tracking', 'direct_post_right_point', 'direct_post_left_point', 'natural_pre_right_point', 'natural_pre_left_point', 'natural_left_tracking', 'natural_right_tracking', 'natural_post_right_point', 'natural_post_left_point']



Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_pre_right_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_pre_left_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_left_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S1-averted_right_tracking_raw.fif
[done]
Writing /hpc

...

Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_pre_left_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_pre_right_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_right_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-averted_left_tracking_raw.fif
[done]
Writing /hpc

Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_pre_left_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_pre_right_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_right_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-natural_left_tracking_raw.fif
[done]
Writing /hpc

Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_pre_left_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_pre_right_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_right_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data/EEG-S7-direct_left_tracking_raw.fif
[done]
Writing /hpc/igum002

Saving process...: 18it [00:00, 57.05it/s]
In progress:  83%|████████▎ | 10/12 [00:15<00:03,  1.54s/it]


IndexError: list index out of range

In [120]:
import os
path_2_csv_files = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG"
list_file_names = []
full_path_4_each_file = []
for (root, dirs, file) in os.walk(path_2_csv_files):
    for f in file:
        if ".csv" in f:
            # Populate all file names only
            list_file_names.append(f)
            list_file_names.sort()

            # Populate all full path of filenames
            full_path_4_each_file.append(os.path.join(root, f))
            full_path_4_each_file.sort()

    print(list_file_names)
    print(full_path_4_each_file)
    

['EEG-S1.csv', 'EEG-S2.csv', 'EEG-S3.csv', 'EEG-S4.csv', 'EEG-S5.csv', 'EEG-S6.csv', 'EEG-S7.csv', 'EEG-S8.csv']
['/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S1.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S2.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S3.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S4.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S5.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S6.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S7.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S8.csv']


In [104]:
list_file_names.sort()
list_file_names

['EEG-S1.csv',
 'EEG-S2.csv',
 'EEG-S3.csv',
 'EEG-S4.csv',
 'EEG-S5.csv',
 'EEG-S6.csv',
 'EEG-S7.csv',
 'EEG-S8.csv']

In [89]:
# stringA = filenames_fif_baseline[0]
stringA = "/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S10-averted_baseline_raw.fif"
print(stringA)

# Match pattern EEG-Sx (x = any number)
regex = r"\D{3}-S\d+"


test_file_name = re.search(regex,stringA)
print(test_file_name)
print("Start index : ", test_file_name.start())
print("End index : ", test_file_name.end())
print(type(stringA[test_file_name.start():test_file_name.end()]))



/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S10-averted_baseline_raw.fif
<re.Match object; span=(48, 55), match='EEG-S10'>
Start index :  48
End index :  55
<class 'str'>


Work in progress saving into file name (success)

In [20]:
print(filenames_fif_baseline[0]) # Recheck the file name
# baseline_data_needs_label.save("baseline_test_saving_raw.fif", overwrite=True)


/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S1-averted_baseline_raw.fif


In [47]:
# %% Get the first 12 markers' indices and extract the data
indicesofBaselineMarkers = indicesOfMarkers[:13]

# Get the 1st and 13th index and chunk dataframe based on those indices, and convert it into numpy array
baseline_data = df.iloc[indicesofBaselineMarkers[0] : indicesofBaselineMarkers[-1], 1:17].to_numpy() * 1e-6

 # Save the baseline data into .fif file 
# baseline_data = 

# _(Ensure there are 12 markers in the baseline_data )

# Save the data into .fif file 


_Checking there are 12 / 13 markers in baseline data because marker is closed for some files. Right after one another between end and opening marker

In [33]:
# Converting all markers (pandas data frame to list)
markers_bl = baseline_df['Marker'].tolist()
# %%  Find all experimental markers and print them out.

indicesOfMarkers_bl = []  # Empty list to contain indices of markers
for i, c in enumerate(markers_bl):
    if "9999999" in str(c) : #todo untuk data EEG-S1-6.csv, tambahkan " : "
        indicesOfMarkers_bl.append(i) # check if the number of markers = 36
print(len(indicesOfMarkers_bl))
print(indicesOfMarkers_bl)

12
[0, 7075, 7916, 15335, 16428, 23979, 24692, 32234, 32881, 40382, 41043, 48516]


In [40]:
baseline_df = df.iloc[indicesofBaselineMarkers[0] : indicesofBaselineMarkers[-1] + 1]

ORIGINAL SCRIPT

In [ ]:
import mne
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
#%%
def extract_eeg_data(filename, labelsequence):
    try:
        f = open(filename)
        labelsequence = int(labelsequence)

    except IOError as err_filename:
        print("The format of file name is not correct or file doesn't exist \nThe format must be 'EEG-Sx.csv' , x=subject number ")
        raise
    except ValueError as err_integer:
        print("The labelsequence input must be integer : ", err_integer)
        raise

    else:
        if  labelsequence < 1 or labelsequence > 12:
            print("The value for labelsequence parameter is out of range. It must be be between 1 and 12")
            raise IndexError
        else:

            #%% Load the data
            # filename = "EEG-S1.csv"
            fileName = filename  # The format of file name of EEG must be like this
            print("Processing file : " + fileName)
            df = pd.read_csv(fileName, delimiter=',')
            # %% Define columns for raw csv file
            df.columns = ['Index', 'FP1', 'FP2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1',
                          'O2', 'X1', 'X2', 'X3', 'X4', 'X5',
                          'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'Marker']
            # Converting all markers (pandas data frame to list)
            markers = df['Marker'].tolist()
            # %%  Find all experimental markers and print them out.

            indicesOfMarkers = []  # Empty list to contain indices of markers
            for i, c in enumerate(markers):
                if "9999999" in str(c) : #todo untuk data EEG-S1-6.csv, tambahkan " : "
                    indicesOfMarkers.append(i) # check if the number of markers = 36
            try:
                number_markers = len(indicesOfMarkers)
                if number_markers != 48:
                    raise ValueError("The {} file has incorrect number of markers : {} ! It MUST be 36".format(fileName,number_markers))
            except ValueError as err_unmatch_markers:
                print(err_unmatch_markers)
                raise


            # %% Loop the list of labels in sequence
            # todo Create a list of labels with different sequences and put them into a list (list of list)
            # Order = 1 (Averted/Direct/Natural)
            oddOrder1 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point", "direct_pre_right_point", "direct_pre_left_point",
                         "direct_left_tracking", "direct_right_tracking", "direct_post_right_point", "direct_post_left_point",
                         "natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point"]

            evenOrder1 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point", "direct_pre_left_point", "direct_pre_right_point",
                          "direct_right_tracking", "direct_left_tracking", "direct_post_left_point", "direct_post_right_point",
                          "natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point"]

            # Order = 2 (Averted/Natural/Direct)
            oddOrder2 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point", "natural_pre_right_point", "natural_pre_left_point",
                         "natural_left_tracking", "natural_right_tracking", "natural_post_right_point", "natural_post_left_point",
                         "direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point"]

            evenOrder2 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point", "natural_pre_left_point", "natural_pre_right_point",
                          "natural_right_tracking", "natural_left_tracking", "natural_post_left_point", "natural_post_right_point",
                          "direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point"]

            # Order = 3 (Direct / Natural / Averted)
            oddOrder3 = ["direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point", "natural_pre_right_point", "natural_pre_left_point",
                         "natural_left_tracking", "natural_right_tracking", "natural_post_right_point", "natural_post_left_point",
                         "averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point"]

            evenOrder3 = ["direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point", "natural_pre_left_point", "natural_pre_right_point",
                          "natural_right_tracking", "natural_left_tracking", "natural_post_left_point", "natural_post_right_point",
                          "averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point"]

            # Order = 4 (Direct/Averted/Natural)
            oddOrder4 = ["direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point", "averted_pre_right_point", "averted_pre_left_point",
                         "averted_left_tracking", "averted_right_tracking", "averted_post_right_point", "averted_post_left_point",
                         "natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point"]

            evenOrder4 = ["direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point", "averted_pre_left_point", "averted_pre_right_point",
                          "averted_right_tracking", "averted_left_tracking", "averted_post_left_point", "averted_post_right_point",
                          "natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point"]

            # Order = 5 (Natural/Direct/Averted)
            oddOrder5 = ["natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point", "direct_pre_right_point", "direct_pre_left_point",
                         "direct_left_tracking", "direct_right_tracking", "direct_post_right_point", "direct_post_left_point",
                         "averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point"]

            evenOrder5 = ["natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point", "direct_pre_left_point", "direct_pre_right_point",
                          "direct_right_tracking", "direct_left_tracking", "direct_post_left_point", "direct_post_right_point",
                          "averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point"]

            # Order = 6 (Natural/Averted/Direct)
            oddOrder6 = ["natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point", "averted_pre_right_point", "averted_pre_left_point",
                         "averted_left_tracking", "averted_right_tracking", "averted_post_right_point", "averted_post_left_point",
                         "direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point"]

            evenOrder6 = ["natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point", "averted_pre_left_point", "averted_pre_right_point",
                          "averted_right_tracking", "averted_left_tracking", "averted_post_left_point", "averted_post_right_point",
                          "direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point"]
            #%% Add all labels into a list

            listOfOrders = []
            for i in progressbar(range(6)):
                listOfOrders.append(eval('oddOrder' + str(i+1)))
                listOfOrders.append(eval('evenOrder' + str(i+1)))
            print("Data have been extracted from : " + fileName)
            chosenOrder = listOfOrders[labelsequence-1]
            # %% Chunk the data based on opening and closing markers and get only the 16 channels data (columns)
            chunkedData = []
            for i in range(0, 36, 2):
                # averted_pre_right_point = df.iloc[indicesOfMarkers[0] : indicesOfMarkers[1] +1, 1:17]
                # Change into numpy and convert it from uV (microvolts) / nV to V (volts)
                chunkedData.append(df.iloc[indicesOfMarkers[i]: indicesOfMarkers[i + 1] + 1, 1:17].to_numpy() * 1e-6)
            # %% Load each eye condition file into MNE Python
            # Create 16 channels montage 10-20 international standard
            montage = mne.channels.make_standard_montage('standard_1020')
            # % Create info
            # Pick only 16 channels that are used in Cyton+Daisy OpenBCI
            ch_names = ['FP1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1', 'O2']
            ch_types = ['eeg'] * 16
            info = mne.create_info(
                ch_names=ch_names,
                sfreq=125,
                ch_types=ch_types)
            info.set_montage('standard_1020', match_case=False)
            # %% Create filenames for *.fif based on the sequence of labels above
            filenames_fif = []
            for i in chosenOrder:
                filenames_fif.append(fileName[4:fileName.index(".")] + "-" + i + "_raw.fif")
            #%% Save into *.fif files
            for i, val in enumerate(chunkedData):
                data_need_label = mne.io.RawArray(val.transpose(), info, verbose=False)
                data_need_label.save(join(filenames_fif[i]), overwrite=True)
            # todo save it into MNE-BIDS format



# extract_eeg_data("EEG-S10.csv", 1)
